# Example subclass for writing NIFTI files

### Import the NIFTIWriter class created in READII along with other necessary imports

In [ ]:
from readii.io.writers.nifti_writer import NIFTIWriter
from readii.io.writers.base_writer import BaseWriter
from pathlib import Path
import subprocess
import SimpleITK as sitk
import pandas as pd
import uuid
import random
import sys
from readii.utils import logger

### Define a writer subclass for writing .csv files

In [ ]:
# copy this writer from the other notebook:
class CSVWriter(BaseWriter): # noqa

  # The save method is the only method that needs to be implemented for the subclasses of BaseWriter
  def save(self, data: list, **kwargs) -> Path:  # noqa
    output_path = self.resolve_path(**kwargs)
    with output_path.open('w') as f: # noqa
      pd.DataFrame(data).to_csv(f, index=False)
    return output_path


### Show how the NIFTI Writer can be used on SimpleITK images

In [5]:
ROOT_DIRECTORY = Path("TRASH", "writer_examples", "nifti_writer_examples")
FILENAME_FORMAT = "PatientID-{PatientID}/Study-{Study}/{Modality}/{Modality}_SeriesUID-{SeriesUID}"

data_sets = []
random.seed(42)  # Set random seed for reproducibility

random_5d = lambda: random.randint(10000, 99999)

# Set up some dummy images to save as NIFTI files
for MODALITY in ["CT", "RTSTRUCT"]:
  data_sets.extend([
    {
      "image": sitk.Image(10, 10, 10, sitk.sitkInt16),
      "metadata": pd.DataFrame({"PatientID": ["JohnAdams"], "Study": ["Study001"]}),
      "PatientID": "JohnAdams",
      "Study": "Study001",
      "Modality": MODALITY,
      "SeriesUID": random_5d(),
    },
    {
      "image": sitk.Image(20, 20, 20, sitk.sitkInt16),
      "metadata": pd.DataFrame({"PatientID": ["JaneDoe"], "Study": ["Study002"]}),
      "PatientID": "JaneDoe",
      "Study": "Study002",
      "Modality": MODALITY,
      "SeriesUID": random_5d(),
    },
    {
      "image": sitk.Image(30, 30, 30, sitk.sitkInt16),
      "metadata": pd.DataFrame({"PatientID": ["AliceSmith"], "Study": ["Study003"]}),
      "PatientID": "AliceSmith",
      "Study": "Study003",
      "Modality": MODALITY,
      "SeriesUID": random_5d(),
    }
  ])

# Create a writer with the specified root directory and filename format
with (
  NIFTIWriter(
    root_directory=ROOT_DIRECTORY, 
    filename_format=f"{FILENAME_FORMAT}.nii.gz",
    overwrite=True
  ) as nifti_writer,
  CSVWriter(
    root_directory=ROOT_DIRECTORY, 
    filename_format=f"{FILENAME_FORMAT}_metadata.csv"
  ) as metadata_writer
):
  # Iterate over the data sets and save them
  for data_set in data_sets:

    # The actual data being saved is image or data, but the rest of the kwargs are 
    # only for resolving the filename
    try:
      nifti_writer.save(
        image=data_set["image"],
        PatientID=data_set["PatientID"],
        Study=data_set["Study"],
        Modality=data_set["Modality"],
        SeriesUID=data_set["SeriesUID"]
      )
      metadata_writer.save(
        data=data_set["metadata"],
        PatientID=data_set["PatientID"],
        Study=data_set["Study"],
        Modality=data_set["Modality"],
        SeriesUID=data_set["SeriesUID"]
      )
    except FileExistsError as e:
      logger.exception(f"Error saving data set: {e}")
      sys.exit(1)

output = subprocess.check_output(["tree", "-nF", ROOT_DIRECTORY])
print(output.decode("utf-8"))

TRASH/writer_examples/nifti_writer_examples/
├── PatientID-AliceSmith/
│   └── Study-Study003/
│       ├── CT/
│       │   ├── CT_SeriesUID-13278.nii.gz
│       │   └── CT_SeriesUID-13278_metadata.csv
│       └── RTSTRUCT/
│           ├── RTSTRUCT_SeriesUID-39256.nii.gz
│           └── RTSTRUCT_SeriesUID-39256_metadata.csv
├── PatientID-JaneDoe/
│   └── Study-Study002/
│       ├── CT/
│       │   ├── CT_SeriesUID-24592.nii.gz
│       │   └── CT_SeriesUID-24592_metadata.csv
│       └── RTSTRUCT/
│           ├── RTSTRUCT_SeriesUID-42098.nii.gz
│           └── RTSTRUCT_SeriesUID-42098_metadata.csv
└── PatientID-JohnAdams/
    └── Study-Study001/
        ├── CT/
        │   ├── CT_SeriesUID-93810.nii.gz
        │   └── CT_SeriesUID-93810_metadata.csv
        └── RTSTRUCT/
            ├── RTSTRUCT_SeriesUID-46048.nii.gz
            └── RTSTRUCT_SeriesUID-46048_metadata.csv

13 directories, 12 files

